### Imports

In [15]:
from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT
import time
from collections import defaultdict
import numpy as np

### Testing with random actions

In [13]:
env = gym_tetris.make('TetrisA-v2')
env = JoypadSpace(env, MOVEMENT)
env.reset()

for _ in range(1000):
    env.render()  # Render the environment
    action = env.action_space.sample()  # Sample a random action
    obs, reward, done, info = env.step(action)  # Take a step in the environment
    if done:
        print("Game Over! Resetting the environment.")
        env.reset()  # Reset the environment if done

env.close()  # Close the environment

c:\Users\sguir\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment TetrisA-v2 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
c:\Users\sguir\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
c:\Users\sguir\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(
c:\Users\sguir\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:219: DeprecationW

In [12]:
env.close()

### Sarsa

In [22]:
class SARSA(object):
    def __init__(self, env, alpha, epsilon, gamma, timeout):
        # definte the parameters
        self.alpha = alpha 
        self.epsilon = epsilon
        self.gamma = gamma

        # environment
        self.env = env

        # define the Q value table
        self.Q = defaultdict(lambda: defaultdict(float))

        # define the timeout for a single episode
        self.timeout = timeout

    def behavior_policy(self, state):

        # epsilon-greedy policy
        if np.random.rand() < self.epsilon:
            
            # explore
            return self.env.action_space.sample()
        
        else:

            # exploit
            return np.argmax(self.Q[state])
        
    def update(self, s, a, r, s_prime, a_prime):

        # update the Q value table using the SARSA update rule
        self.Q[s][a] += self.alpha * (r + self.gamma * self.Q[s_prime][a_prime] - self.Q[s][a])

    def train(self, num_episodes):

        # initialize rewards
        rewards = []

        # train the agent for a number of episodes
        for episode in range(num_episodes):

            # reset the environment
            state = self.env.reset()

            # get the first action
            action = self.behavior_policy(state)

            # initialize the reward
            reward = 0

            for t in range(self.timeout):

                # take the action
                next_state, reward, done, info = self.env.step(action)

                # get the next action
                next_action = self.behavior_policy(next_state)

                # update the Q value table
                self.update(state, action, reward, next_state, next_action)

                # update the state, action, and reward
                state = next_state
                action = next_action
                reward += reward

                # if the game is over, reset the environment
                if done:
                    break

            # append the reward to the list of rewards
            rewards.append(reward)

            # print the average reward for the episode
            avg_reward = np.mean(rewards[-100:])
            print(f"Episode {episode + 1}/{num_episodes} - Average Reward: {avg_reward:.2f}")

            # print the episode number
            print(f"Episode {episode + 1}/{num_episodes} completed.")

In [24]:
# Example usage
if __name__ == "__main__":
    env = gym_tetris.make('TetrisA-v2')
    env = JoypadSpace(env, MOVEMENT)
    env.reset()

    # Define the parameters
    alpha = 0.1
    epsilon = 0.1
    gamma = 0.9
    timeout = 1000

    # Create the SARSA agent
    agent = SARSA(env, alpha, epsilon, gamma, timeout)

    # Train the agent
    agent.train(num_episodes=1000)

    env.close()

TypeError: unhashable type: 'numpy.ndarray'